설치가 필요한 라이브러리

In [ ]:
!pip install SimpleITK
!pip install numpy
!pip install pydicom

##### 경로용함수

In [2]:
import os
import SimpleITK as sitk
import numpy as np
import pydicom

def set_base(target_type):
    """
    Parameter:
        target_type : 'nii' or 'dicom'
    """
    #target_type = 'nii'
    base_path = os.path.join('.', 'example_data', target_type + '_example')
    return base_path

### Pydicom을 이용한 의료 이미지 다루기

#### dicom 파일 읽고 쓰기

dicom 파일은 폴더 째로 저장된 경우가 많으며,   
해당 폴더에 저장된 이미지가 합쳐져서 하나의 3d 이미지를 구성할 수 있다.   

1. 실제 픽셀에 어떤 값이 저장되는지에 대한 데이터 ex) (0,0,0) 픽셀의 값은 -1024
2. 해당 dicom 파일의 값의 메타데이터 ex) 환자 정보, 각 픽셀의 물리적 공간

등이 저장되어있다

예제 데이터는 public data로 모두에게 공개된 상태이기에, 환자의 개인 정보가 아닌   
데이터를 구분하는 id가 적힌 모습을 볼 수 있다

In [15]:
base_path = set_base('dicom')
def dicom_path_gen(folder):
    result_path = os.path.realpath(os.path.join(base_path, folder))
    return result_path

artery_path = dicom_path_gen('APhase')
delay_path = dicom_path_gen('DPhase')
non_contrast_path = dicom_path_gen('NPhase')
seg_path = os.path.join(dicom_path_gen('Segmentation'), '1-1.dcm')


artery_files = [f for f in os.listdir(artery_path) if f.lower().endswith(".dcm")]
# 첫 번째 DICOM 파일 선택
dicom_path = os.path.join(artery_path, artery_files[0])
print(f"첫 번째 DICOM 파일: {dicom_path}")

# DICOM 파일 읽기
dicom_data = pydicom.dcmread(dicom_path)
# ✅ DICOM 헤더 확인
print("\n📌 DICOM 헤더 정보")
print(f"Series: {dicom_data.SeriesNumber}")
print(f"Instance: {dicom_data.InstanceNumber}")
print(f"Patient Name: {dicom_data.PatientName}")
print(f"Patient ID: {dicom_data.PatientID}")
print(f"Modality: {dicom_data.Modality}")
print(f"Study Date: {dicom_data.StudyDate}")
print(f"Rows x Columns: {dicom_data.Rows} x {dicom_data.Columns}")

첫 번째 DICOM 파일: /home/seongwoo/seong_test/code/medical_image_education/example_data/dicom_example/APhase/1-130.dcm

📌 DICOM 헤더 정보
Series: 7
Instance: 130
Patient Name: KiTS-00000
Patient ID: KiTS-00000
Modality: CT
Study Date: 20030629
Rows x Columns: 512 x 512


아까 Dcm파일은 폴더 당 하나의 파일로 저장된 경우가 많다고 표현했다.   
이 말의 자세한 뜻은 : "Dicom 파일은 한 폴더에 Volume을 여러개의 이미지로 나누어져 저장이 된다는 뜻이다."
그렇다면 이렇게 여러개로 나누어진 이미지는 어떻게 합칠 수 있을까?

위의 Header를 나열한 것을 보면 Series와, Instance가 있다.

이를 자세히 보기 위해 정렬을 시켜 살펴보자

In [24]:
sorted_artery_files = sorted(artery_files)

for idx, filename in enumerate(sorted_artery_files):
    if idx > 3 and idx < (len(sorted_artery_files) - 3):
        continue

    dicom_path = os.path.join(artery_path, filename)
    # DICOM 파일 읽기
    dicom_data = pydicom.dcmread(dicom_path)
    # ✅ DICOM 헤더 확인
    print(f"{idx} 번째 DICOM 파일: {dicom_path}")
    print("📌 DICOM 헤더 정보")
    print(f"Series: {dicom_data.SeriesNumber}")
    print(f"Instance: {dicom_data.InstanceNumber}\n\n")
    


0 번째 DICOM 파일: /home/seongwoo/seong_test/code/medical_image_education/example_data/dicom_example/APhase/1-001.dcm
📌 DICOM 헤더 정보
Series: 7
Instance: 1


1 번째 DICOM 파일: /home/seongwoo/seong_test/code/medical_image_education/example_data/dicom_example/APhase/1-002.dcm
📌 DICOM 헤더 정보
Series: 7
Instance: 2


2 번째 DICOM 파일: /home/seongwoo/seong_test/code/medical_image_education/example_data/dicom_example/APhase/1-003.dcm
📌 DICOM 헤더 정보
Series: 7
Instance: 3


3 번째 DICOM 파일: /home/seongwoo/seong_test/code/medical_image_education/example_data/dicom_example/APhase/1-004.dcm
📌 DICOM 헤더 정보
Series: 7
Instance: 4


608 번째 DICOM 파일: /home/seongwoo/seong_test/code/medical_image_education/example_data/dicom_example/APhase/1-609.dcm
📌 DICOM 헤더 정보
Series: 7
Instance: 609


609 번째 DICOM 파일: /home/seongwoo/seong_test/code/medical_image_education/example_data/dicom_example/APhase/1-610.dcm
📌 DICOM 헤더 정보
Series: 7
Instance: 610


610 번째 DICOM 파일: /home/seongwoo/seong_test/code/medical_image_education/example_

또한 dicom 파일을 array로 바꾸어보자

In [13]:
image_array = dicom_data.pixel_array

이런 식으로 Dicom 파일을 하나 읽어서 바꿔보면, 3D Volume이 아닌 각각의 Frame이 읽히는 모습을 볼 수 있다.

In [14]:
print(image_array.shape)

(512, 512)


##### 모든 태그를 한번 봐보고 싶다면

In [7]:
# ✅ 전체 메타데이터 출력
print("\n📌 모든 DICOM 메타데이터")
for element in dicom_data:
    print(f"{element.keyword}: {element.value}")


📌 모든 DICOM 메타데이터
SpecificCharacterSet: ISO_IR 100
ImageType: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
SOPClassUID: 1.2.840.10008.5.1.4.1.1.2
SOPInstanceUID: 1.3.6.1.4.1.14519.5.2.1.6919.4624.263098720532974756124207703389
StudyDate: 20030629
SeriesDate: 20030629
AcquisitionDate: 20030629
ContentDate: 20030629
AcquisitionDateTime: 20030629083431.119000
StudyTime: 081517.464000
SeriesTime: 084056.727000
AcquisitionTime: 083431.119000
ContentTime: 083431.119000
AccessionNumber: 
Modality: CT
Manufacturer: SIEMENS
ReferringPhysicianName: 
StudyDescription: three_phase__abdomen
ProcedureCodeSequence: [(0008,0100) Code Value                          SH: 'IMG240'
(0008,0102) Coding Scheme Designator            SH: 'LOCAL'
(0008,0104) Code Meaning                        LO: 'CT ABDOMEN/PELVIS ANGIO WO & W CONTRAST']
SeriesDescription: arterial
ManufacturerModelName: SOMATOM Definition Flash
ReferencedStudySequence: [(0008,1150) Referenced SOP Class UID            UI: Detached Study Man

dicom header는 태그와 데이터로 이루어져있으며, 각 태그가 의미하는 바가 궁금하다면

[dicom 태그 모음집](https://www.dicomlibrary.com/dicom/dicom-tags/)

을 한번 봐보자!(매우 많다)

#### Dicom 변환

### SimpleITK를 이용한 의료 이미지 다루기

#### dicom

dicom 파일은 대부분 폴더 당 하나의 파일이라고 생각하면 편하다   
(한 파일에 저장된 경우도 있긴하다!)

한 파일만 읽어들인 경우

In [3]:
def dicom_path_gen(folder):
    result_path = os.path.realpath(os.path.join(base_path, folder))
    return result_path

artery_path = dicom_path_gen('APhase')
delay_path = dicom_path_gen('DPhase')
non_contrast_path = dicom_path_gen('NPhase')
seg_path = os.path.join(dicom_path_gen('Segmentation'), '1-1.dcm')

one_artery_path = os.path.join(artery_path, '1-001.dcm')
a_image = sitk.ReadImage(one_artery_path)
print((sitk.GetArrayFromImage(a_image)).shape)

(1, 512, 512)


폴더 째로 읽어들인 경우 : 전체 3d 의료 이미지를 하나로 합친 것을 볼 수 있다!

In [4]:
def dicom_read(folder_path):
    reader = sitk.ImageSeriesReader()
    dicom_series = reader.GetGDCMSeriesFileNames(folder_path)
    reader.SetFileNames(dicom_series)
    result_image = reader.Execute()
    return result_image

a_image = dicom_read(artery_path)
d_image = dicom_read(delay_path)
n_image = dicom_read(non_contrast_path)

seg = sitk.ReadImage(seg_path)

print((sitk.GetArrayFromImage(a_image)).shape)

(611, 512, 512)


sitk로 Dicom을 읽어드린 경우, Dicom Header의 메타 데이터가 사라지고,   
일부 메타데이터만 남게된다

이 남은 메타데이터에 대해선 아래의 nifti 메타데이터를 확인하면서 보자   
간략히 설명하자면, 환자의 정보 병원 정보 등이 사라지고, 실제 이미지를 표현하는 메타데이터만 남는다!

In [17]:
a_array = sitk.GetArrayFromImage(a_image)
print(a_array.shape)
print(seg)


(611, 512, 512)
Image (0x2f43810)
  RTTI typeinfo:   itk::Image<unsigned char, 3u>
  Reference Count: 1
  Modified Time: 33269
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 33252
  UpdateMTime: 33265
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 1222]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 1222]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 1222]
  Spacing: [1, 1, 1]
  Origin: [-243.04, -409.54, -975]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
1 0 0
0 1 0
0 0 1

  PointToIndexMatrix: 
1 0 0
0 1 0
0 0 1

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x85c90c0)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, unsigned char>
      Reference Count: 1

#### nifti

##### 기초

nii.gz 파일은 SimpleITK 라이브러리를 통해서 읽고, 수정이 가능하다   
ReadImage를 통해 nii파일을 읽고, WriteImage를 통해 저장이 가능하다

In [36]:
base_path = set_base('nii')
cect_path = os.path.join(base_path, 'cect_example.nii.gz')
ncct_path = os.path.join(base_path, 'ncct_example.nii.gz')
seg_path = os.path.join(base_path, 'label_example.nii.gz')

cect_image = sitk.ReadImage(cect_path)
ncct_image = sitk.ReadImage(ncct_path)
seg_image = sitk.ReadImage(seg_path)

save_path = os.path.join('.', 'temp_save.nii.gz')
sitk.WriteImage(cect_image, save_path)

Read Image를 하면 skimage 형식으로 저장이 되며,   
이 데이터에는 nii파일의 meta데이터와 동시에 실제 voxel별 값을 저장하는 Array가 존재한다

In [8]:
print(cect_image)

Image (0x139996d0)
  RTTI typeinfo:   itk::Image<float, 3u>
  Reference Count: 1
  Modified Time: 1837
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 1812
  UpdateMTime: 1833
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 56]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 56]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 56]
  Spacing: [0.925781, 0.925781, 5]
  Origin: [-247.537, -411.037, -1079.7]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.925781 0 0
0 0.925781 0
0 0 5

  PointToIndexMatrix: 
1.08017 0 0
0 1.08017 0
0 0 0.2

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x13997910)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, float>
      Reference 

주로 사용하는 MetaData는 아래와 같이 있다   
1. Spacing, Origin, Direction
2. PixelID    

이는 실제로 저장되는 key값과는 다르며, 라이브러리에서 보기 쉽게 출력해주는 것이다.

[실제 Nifti의 metadata 저장방식 참고자료](https://brainder.org/2012/09/23/the-nifti-file-format/)

##### Array

GetArrayFromImage 함수를 통해 실제 저장된 Array를 불러올 수 있다.

In [29]:
cect_array = sitk.GetArrayFromImage(cect_image)
print(cect_array)

[[[-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  ...
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]]

 [[-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  ...
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]]

 [[-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  ...
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -1024. ... -1024. -1024. -1024.]]

 ...

 [[-1024. -1024. -1024. ... -1024. -1024. -1024.]
  [-1024. -1024. -10

##### Spacing, Origin, Direction   


아래의 3가지 메타데이터는 ndarray로 구성된 픽셀 공간 -> 실제 물리적인 공간으로, 변환을 수행해줄 때   
사용되는 메타데이터이다.   
   
해당 데이터들이 동일하지 않을 때, 저장된 array의 값이 같더라도, 저장되는 값의 의미는 달라진다

In [18]:
print(f"원본 이미지의 메타데이터 \n\
      spacing : {cect_image.GetSpacing()} \n\
      origin : {cect_image.GetOrigin()} \n\
      Direction : {cect_image.GetDirection()}")

원본 이미지의 메타데이터 
      spacing : (0.92578125, 0.92578125, 5.0) 
      origin : (-247.537109375, -411.037109375, -1079.699951171875) 
      Direction : (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


GetArrayFromImage와 GetImageFromArray 함수를 통해 내부의 배열은 같은채   
메타데이터가 기본값인 temp_image를 생성해보자

In [19]:
temp_array = sitk.GetArrayFromImage(cect_image)
temp_image = sitk.GetImageFromArray(temp_array)

print(f"생성된 이미지의 메타데이터 \n\
      spacing : {temp_image.GetSpacing()} \n\
      origin : {temp_image.GetOrigin()} \n\
      Direction : {temp_image.GetDirection()}")

생성된 이미지의 메타데이터 
      spacing : (1.0, 1.0, 1.0) 
      origin : (0.0, 0.0, 0.0) 
      Direction : (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


이런 식으로 metadata가 다른 경우 실질적으로 의미하는 값이 다르기에,   
Array를 통해 이미지를 만드려고 할 땐, 메타데이터를 복사해야한다   

이를 수행해주는 것이 CopyInformation()이다.

In [20]:
temp_image.CopyInformation(cect_image)
print(f"생성된 이미지의 메타데이터 \n\
      spacing : {temp_image.GetSpacing()} \n\
      origin : {temp_image.GetOrigin()} \n\
      Direction : {temp_image.GetDirection()}")

생성된 이미지의 메타데이터 
      spacing : (0.92578125, 0.92578125, 5.0) 
      origin : (-247.537109375, -411.037109375, -1079.699951171875) 
      Direction : (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


##### PixelID

해당 메타데이터는 자주 사용하지는 않는다.   
이는 nii.gz파일에서 픽셀값을 저장하는 데이터 형식을 다룰 때 사용된다   
주로 데이터를 보낼 때 의료데이터의 값이 크기에, 압축을 수행할 때 사용된다

각 Pixel ID가 의미하는 데이터 형식이 궁금하다면, 기초에 있는 링크를 참조하자

In [22]:
print(cect_image.GetPixelID())

8


위의 예시에서 CECT Image는 PixelID = 8 즉, signed int 형식으로 한 픽셀당 32bits를 통해 저장이 된다.   
이를 압축시켜 보자

In [30]:
#Clip Image Intenstiy 0 to 255
clipped_array = np.clip(cect_array, 0, 255)
output_image = sitk.GetImageFromArray(clipped_array)
output_image.CopyInformation(cect_image)

zip_image_uint8 = sitk.Cast(output_image, sitk.sitkUInt8)
print(zip_image_uint8.GetPixelID())

sitk.WriteImage(zip_image_uint8, 'zip_file.nii.gz')

1


자 이제 두 파일의 용량을 비교해보자
원본 파일의 intensity를 clip하고,   
저장형식을 바꾸자, 한 파일 당 18mb였던 파일이 2mb로 준 모습을 확인할 수 있다   

(리눅스 기준, 윈도우라면 파일을 속성으로 확인해보자)

In [31]:
!du -sh ./example_data/nii_example/cect_example.nii.gz
!du -sh ./zip_file.nii.gz

18M	./example_data/nii_example/cect_example.nii.gz
2.0M	./zip_file.nii.gz
